In [ ]:
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import copy


In [ ]:
today  = date.today()
yymmdd = today.strftime("%Y%m%d")

#/glade/p/cesmdata/cseg/inputdata/ocn/docn7/SOM/pop_frc.b.e20.B1850.f09_g17.pi_control.all.297.20180523.nc
#/glade/p/cesmdata/cseg/inputdata/ocn/docn7/SOM/mom_frc_b.cesm3_cam058_mom_e.B1850WscMOM.ne30_L58_t061.camdev_cice5.026c_50-99_c20221111.nc

case="cesm1"
ifi1="/glade/p/cesmdata/cseg/inputdata/ocn/docn7/SOM/pop_frc.b.e11.B1850LENS.f09_g16.pi_control.002.20190923.nc"
ifi2="/glade/p/cesmdata/cseg/inputdata/ocn/docn7/SOM/pop_frc.b.e20.B1850.f09_g17.pi_control.all.297.20180523.nc"
ofi="pop_frc.b.e20.B1850.f09_g17.NH-CESM1.SH-CES2."+yymmdd+".nc"

sm1=xr.open_dataset(ifi1)
xc1=sm1['xc'].values
yc1=sm1['yc'].values

sm2=xr.open_dataset(ifi2)
xc2=sm1['xc'].values
yc2=sm1['yc'].values

smx=copy.deepcopy(sm1)


print(list(sm1.variables))

print(np.shape(sm1['area']))




hbl1=sm1['hblt']
qdp1=sm1['qdp']
are1=sm1['area']
msk1=sm1['mask']
hbl2=sm2['hblt']
qdp2=sm2['qdp']
are2=sm2['area']
msk2=sm2['mask']

qdpx = copy.deepcopy( qdp1 )
hblx = copy.deepcopy( hbl1 )


moo=0*msk1
moo=np.where( msk1==0, 0., 1.)

plt.contourf( moo )
plt.colorbar()

for imo in np.arange(12):
    qm1=qdp1[imo,:,:]
    qm2=qdp2[imo,:,:]
    qdpx[imo,:,:] = np.where( yc1>=0., qm1, qm2)

for imo in np.arange(12):
    hm1=hbl1[imo,:,:]
    hm2=hbl2[imo,:,:]
    hblx[imo,:,:] = np.where( yc1>=0., hm1, hm2)

#aq=qdp1[0,:,:]*are1*moo
#aq=qm1*are1*moo
#print("Global sum - CESM1", np.sum(aq)/np.sum(are1*moo) )
#aq=qm2*are1*moo
#print("Global sum - CESM2", np.sum(aq)/np.sum(are1*moo) )
#aq=qmx*are1*moo
#print("Global sum - SWAP", np.sum(aq)/np.sum(are1*moo) )

#moo[:,:]=1.0
qwa1=np.average( qdp1 , axis=0)
aq=qwa1*are1*moo
print("Global sum - CESM1", np.sum(aq)/np.sum(are1*moo) )
qwa2=np.average( qdp2 , axis=0)
aq=qwa2*are1*moo
print("Global sum - CESM2", np.sum(aq)/np.sum(are1*moo) )
qwa=np.average( qdpx , axis=0)
aq=qwa*are1*moo
print("Global sum - SWAP", np.sum(aq)/np.sum(are1*moo) )

qdpx = qdpx - np.sum(aq)/np.sum(are1*moo) 
qwa=np.average( qdpx , axis=0)
aq=qwa*are1*moo
print("Global sum - SWAP rezeroed", np.sum(aq)/np.sum(are1*moo) )

clevels=np.linspace(-100.,100.,num=31)
fig = plt.figure(figsize=(20, 9))
ax = fig.add_subplot(1, 3, 1 ) 
cf=ax.contourf( qwa1 , levels=clevels )
plt.colorbar(cf,ax=ax,shrink=0.6)
ax = fig.add_subplot(1, 3, 2 ) 
cf=ax.contourf( qwa2 , levels=clevels )
plt.colorbar(cf,ax=ax,shrink=0.6)
ax = fig.add_subplot(1, 3, 3 ) 
cf=ax.contourf( qwa , levels=clevels )
plt.colorbar(cf,ax=ax,shrink=0.6)

clevels=np.linspace(0.,100.,num=31)
fig = plt.figure(figsize=(20, 9))
ax = fig.add_subplot(1, 3, 1 ) 
cf=ax.contourf( hbl1[0,:,:] , levels=clevels )
plt.colorbar(cf,ax=ax,shrink=0.6)
ax = fig.add_subplot(1, 3, 2 ) 
cf=ax.contourf( hbl2[0,:,:] , levels=clevels )
plt.colorbar(cf,ax=ax,shrink=0.6)
ax = fig.add_subplot(1, 3, 3 ) 
cf=ax.contourf( hblx[0,:,:] , levels=clevels )
plt.colorbar(cf,ax=ax,shrink=0.6)


In [ ]:
smx['hblt']=hblx
smx['qdp']=qdpx
smx.to_netcdf( ofi )

In [ ]:
flds0=list( sm.variables )
flds=[]

for ifld in flds0:
    if sm[ifld].ndim==3:
        flds.append(ifld)

""" 
Unclear whether we should zonally
average all fields. Maybe just
hblt and qdp
"""
flds.remove('dhdx')
flds.remove('dhdy')
flds.remove('S')
flds.remove('T')
flds.remove('U')
flds.remove('V')